In [45]:
import pandas as pd, numpy as np, json, os
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
plt.style.use('seaborn-whitegrid')

In [47]:
#!pip install d3IpyPlus
from d3IpyPlus import ScatterPlot, LinePlot, BarPlot, StackedArea

In [48]:
#!pip install pyecharts

In [397]:
m=json.loads(open('ignore/member_timelines.json','r').read())
members=json.loads(open('ignore/member_simple.json','r').read())

In [398]:
d=pd.DataFrame(members['deaths'],index=['v']).T

In [399]:
d['v']=' †'

In [400]:
s=pd.DataFrame(members['s2'],index=['s']).T
s=s.join(d)
s['v']=s['v'].fillna('')
s['s']=s['s']+s['v']
s=s[['s']]
s['w']=s['s'].str.split(' ').str[0].str[0]+s['s'].str.split(' ').str[-1].str[0]

In [401]:
szd_color='#E91C62'

In [402]:
pcolors=json.loads(open('json/pcolors.json','r').read())

In [403]:
g=pd.DataFrame(members['megye'],index=['Megye ']).T
g.head()

,Megye
Alexandrescu Emil Lucian | 1937-11-19,🇷🇴 Iași
Dobritoiu Teodoru | 1954-06-14,🇷🇴 Galac
Stef Mircea-Ioan | 1930-11-07,🇷🇴 Iași
Barbat Vasile | 1937-10-06,🇷🇴 Temes
Dumitrescu Alexandru | Ismeretlen,🇷🇴 Buzău


In [404]:
data={}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        if i['Csoportok'] not in data:data[i['Csoportok']]={'start':pd.to_datetime('2030'),'end':pd.to_datetime('1980')}
        start=pd.to_datetime(i['start'])
        end=pd.to_datetime(i['end'])
        data[i['Csoportok']]['start']=min(data[i['Csoportok']]['start'],start)
        data[i['Csoportok']]['end']=max(data[i['Csoportok']]['end'],end)

0.0 %
3.3029462280354074 %
6.605892456070815 %
9.908838684106223 %
13.21178491214163 %
16.514731140177037 %
19.817677368212447 %
23.120623596247853 %
26.42356982428326 %
29.72651605231867 %
33.029462280354075 %
36.332408508389484 %
39.635354736424894 %
42.938300964460296 %
46.241247192495706 %
49.544193420531116 %
52.84713964856652 %
56.15008587660193 %
59.45303210463734 %
62.75597833267275 %
66.05892456070815 %
69.36187078874356 %
72.66481701677897 %
75.96776324481438 %
79.27070947284979 %
82.57365570088518 %
85.87660192892059 %
89.179548156956 %
92.48249438499141 %
95.78544061302682 %
99.08838684106223 %


In [405]:
part_timelines=pd.DataFrame(data).T
part_timelines.index.name='id'

In [406]:
part_timelines['start']=part_timelines['start'].astype(str).str[:10]
part_timelines['end']=part_timelines['end'].astype(str).str[:10]

In [407]:
p=part_timelines.reset_index()[['id']]
p['Color']=p['id'].str.split(' ').str[1].str.strip()
p['Part']=p['id'].str.split('-').str[0].str.strip()

In [408]:
part_timelines=part_timelines.join(p.set_index('id').drop_duplicates())

In [409]:
logos={'🕊️':'ALDE', '🌹':'PSD', '📐':'PNL', '🔶':'PD', '⚛️':'PP-DD', '👤':'Független', '⚪️':'FSN',
       '✳️':'PNTCD', '🔱':'PRM', '🌷':'RMDSZ', '🍀':'ECO', '🍏':'PMP',
       '🔷':'USR', '🦅':'UNPR', '🔘':'Más'}

In [410]:
part_timelines['logo']=part_timelines['Part'].str.split(' ').str[0]

In [411]:
part_timelines['group']=[logos[i] for i in part_timelines['logo'].values]
part_timelines['label']=[i+' '+logos[i] for i in part_timelines['logo'].values]

In [412]:
part_timelines=part_timelines.join(pd.DataFrame(pcolors,index=['cvalue']).T,on='group').fillna('grey')

In [414]:
part_timelines_list=list(part_timelines.reset_index().T.to_dict().values())

In [415]:
open('json/part_timelines.json','w').write(json.dumps(part_timelines_list))

9874

Resolve for non-activity


In [450]:
from calendar import monthrange

In [484]:
data={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        part=i['Csoportok']
        if part not in data:data[part]={}
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:7]
            if st not in data[part]:data[part][st]=0
            data[part][st]+=1/monthrange(t.year,t.month)[1]

0.0 %
3.3029462280354074 %
6.605892456070815 %
9.908838684106223 %
13.21178491214163 %
16.514731140177037 %
19.817677368212447 %
23.120623596247853 %
26.42356982428326 %
29.72651605231867 %
33.029462280354075 %
36.332408508389484 %
39.635354736424894 %
42.938300964460296 %
46.241247192495706 %
49.544193420531116 %
52.84713964856652 %
56.15008587660193 %
59.45303210463734 %
62.75597833267275 %
66.05892456070815 %
69.36187078874356 %
72.66481701677897 %
75.96776324481438 %
79.27070947284979 %
82.57365570088518 %
85.87660192892059 %
89.179548156956 %
92.48249438499141 %
95.78544061302682 %
99.08838684106223 %


In [493]:
df=pd.DataFrame(data)

In [494]:
df=df.stack().reset_index()
df.columns=['x','id','y']

In [490]:
df=df.groupby(['id','x']).mean().rolling(7).mean().reset_index().dropna()

In [495]:
df2=df.join(part_timelines,on='id')

In [497]:
df2=df2[df2['y']>5]
values=list(df2.T.to_dict().values())
open('json/part_timelines2.json','w').write(json.dumps(values))

847683